In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# uv run mcp run src/cora_customer_agent/mcp.py

mcp_client = MultiServerMCPClient(
    {
        "company_rag": {
            "transport": "stdio",
            "command": "uv",
            "args": ["run", "mcp", "run", "src/cora_customer_agent/mcp.py"],
        }
    }
)

In [2]:
tools = await mcp_client.get_tools()

In [63]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0.8,
    reasoning=False, 
    num_predict=512,
    num_ctx=10000,
    validate_model_on_init=True, 
    keep_alive=1000
)

In [60]:
system_prompt = """
You are a helpful customer support assistant for TechHive. 
Use the get_company_faq_answers tool for general inquiries about company policies, products, services, or processes like placing orders. 
Use the get_product_informations tool for detailed questions about specific products, features, or specifications. 
Always use the most appropriate tool based on the query.
"""

In [64]:
from langchain.agents import create_agent 
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
)

In [65]:
async for token, metadata in agent.astream(  
    {"messages": [{"role": "user", "content": "Tell me about the product HiveSmart Light Bulb A60"}]},
    config={"configurable": {"thread_id": "1"}},
    stream_mode="messages",
):
    for block in token.content_blocks:
        
        if 'reasoning' in block:
            print(block['reasoning'], end="", flush=True)
        elif 'text' in block:
            print(block['text'], end="", flush=True)

Product: HiveSmart Light Bulb A60
Category: Smart Home
Description: Energy-efficient Wi-Fi LED bulb controllable via the TechHive app or voice assistants.
Price: €19.99
Features: [Dimmable, 16 million colors, Compatible with Alexa and Google Home]Our HiveSmart Light Bulb A60 is a popular and energy-efficient option for those looking to upgrade their lighting game. With its Wi-Fi connectivity, you can control it remotely using the TechHive app or voice assistants like Alexa and Google Home. It's also dimmable, allowing for customizable brightness levels.

One of the key benefits of this bulb is its compatibility with various smart home systems, making it easy to integrate into your existing setup. Plus, it's designed to last, with a lifespan of up to 25,000 hours and an estimated energy consumption of just 9 watts.

If you have any specific questions or want to know more about the features and benefits of the HiveSmart Light Bulb A60, feel free to ask!